In [ ]:
def get_tmp_path():
    _ , path = tempfile.mkstemp(suffix='.h5')
    return path

def save_model(self, outer_model, path=None):
    import tarfile
    model = outer_model['model']
    model_bytesio = outer_model['model_bytesio']
    inner_path = outer_model['path']

    tempf = tempfile.TemporaryDirectory()
    tar = tarfile.open(inner_path, "r:gz")
    for tarinfo in tar:
        print("tarinfo: ", tarinfo)
        if tarinfo.name == 'kerasmodel.py':
            temp_path = os.path.join(tempf.name, tarinfo.name)
            tar.extract(tarinfo, tempf.name)
    tar.close()


    #if not path:
    _ , tar_path = tempfile.mkstemp(suffix='.tar.gz')

    _, weights_path = tempfile.mkstemp(suffix='.h5')
    # saving model weights
    model.save_weights(weights_path)

    tar = tarfile.open(tar_path, "w:gz")
    tar.add(weights_path,'weights.h5')
    tar.add(temp_path,'kerasmodel.py')
    tar.close()
    return tar_path

def save_model_edit(self, model, old_path, new_path):
    import tarfile
    print("save model edit starts")

    tempf = tempfile.TemporaryDirectory()
    tar = tarfile.open(old_path, "r:gz")
    for tarinfo in tar:
        print("tarinfo: ", tarinfo)
        if tarinfo.name == 'kerasmodel.py':
            temp_path = os.path.join(tempf.name, tarinfo.name)
            tar.extract(tarinfo, tempf.name)
    tar.close()

    _, weights_path = tempfile.mkstemp(suffix='.h5')
    model.save_weights(weights_path)

    tar = tarfile.open(new_path, "w:gz")
    tar.add(weights_path, 'weights.h5')
    tar.add(temp_path,'kerasmodel.py')
    tar.close()
    print("save model edit done")
    return new_path

def load_model(path="model"):
    import importlib.util
    import tarfile
    tempf = tempfile.TemporaryDirectory()
    tar = tarfile.open(path, "r:gz")
    print("inside load model")
    for tarinfo in tar:
        print("tarinfo: ", tarinfo)
        tar.extract(tarinfo, tempf.name)
    tar.close()
    
    print("folder: ", tempf.name)
    for filename in os.listdir(tempf.name):
        print("   filename: ", filename)
        if os.path.isdir(os.path.join(tempf.name,filename)):
            for filename_inner in os.listdir(os.path.join(tempf.name,filename)):
                print("      : ", filename_inner)
    
    # m1 = importlib.import_module(os.path.join(tempf.name,'kerasmodel'))
    spec = importlib.util.spec_from_file_location("create_seed_model", 
                                                  os.path.join(tempf.name,'model_scripts', "kerasmodel.py"))
    foo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(foo)
    # foo.MyClass()
    model = foo.create_seed_model(tempf.name)
    model.load_weights(os.path.join(tempf.name, 'weights.h5'))
    outer_model = {}
    outer_model['model'] = model

    return outer_model

def load_model_from_BytesIO(model_bytesio):
    """ Load a model from a BytesIO object. """
    path = get_tmp_path()
    with open(path, 'wb') as fh:
        fh.write(model_bytesio)
        fh.flush()
    
    return load_model(path)

def serialize_model_to_BytesIO(outer_model):
    
    outfile_name = save_model(outer_model)

    from io import BytesIO
    a = BytesIO()
    a.seek(0, 0)
    with open(outfile_name, 'rb') as f:
        a.write(f.read())
    os.unlink(outfile_name)
    return a



In [ ]:
os.path.join('g','h','g')

In [ ]:
import importlib.util

def load_module(tar_ball,script_name,module_name):
    
    #tempf = tempfile.TemporaryDirectory()
    _, tempf = tempfile.mkstemp()
    #tar = tarfile.open(path, "r:gz")
    print("tempf: ", tempf)
    for tarinfo in tar_ball:
        print("tarinfo: ", tarinfo)
        if tarinfo.name == script_name:
            tar.extract(tarinfo, tempf.name)
    tar.close()
    # m1 = importlib.import_module(os.path.join(tempf.name,'kerasmodel'))
    spec = importlib.util.spec_from_file_location(module_name, os.path.join(tempf.name, "kerasmodel.py"))
    foo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(foo)
    

In [ ]:
import tempfile
import tarfile
model_folder = 'model_scripts'

#import model:shell
#spec = importlib.util.spec_from_file_location("create_seed_model", os.path.join(tempf.name, "kerasmodel.py"))
#foo = importlib.util.module_from_spec(spec)
#spec.loader.exec_module(foo)


outer_model = {}
outer_model['model'] = None# kerasmodel()
from io import BytesIO
#outfile_name = tempfile.TemporaryDirectory()
_, outfile_name = tempfile.mkstemp() 

# transform model_folder to a tar.gz
tar = tarfile.open(outfile_name, "w:gz")
for filename in os.listdir(model_folder):
    print("adding: ", filename)
    tar.add(os.path.join(model_folder,filename))
tar.close()

# tar to bytesio
a = BytesIO()
a.seek(0, 0)
with open(outfile_name, 'rb') as f:
    a.write(f.read())
outer_model['model_bytesio'] = load_model_from_BytesIO(a.getbuffer())

In [ ]:
def save_model(self, outer_model, path=None):
    import tarfile
    model = outer_model['model']
    model_bytesio = outer_model['model_bytesio']
    inner_path = outer_model['path']

    tempf = tempfile.TemporaryDirectory()
    tar = tarfile.open(inner_path, "r:gz")
    for tarinfo in tar:
        print("tarinfo: ", tarinfo)
        if tarinfo.name == 'kerasmodel.py':
            temp_path = os.path.join(tempf.name, tarinfo.name)
            tar.extract(tarinfo, tempf.name)
    tar.close()

In [ ]:
import crazy_test
import wonderkid
crazy_test.jumpboy(wonderkid)

In [2]:
import tarfile
import sys
import tempfile
path = '/Users/mattiasakesson/Documents/projects/fedn/test/fedbird/seed/birdcage'
tempf = tempfile.TemporaryDirectory()
tar = tarfile.open(path, "r:gz")
for tarinfo in tar:
    print("tarinfo: ", tarinfo)
    tar.extract(tarinfo, tempf.name)
tar.close()
sys.path.append(tempf.name)
from kerasmodel import create_seed_model

tarinfo:  <TarInfo 'weights.h5' at 0x10c5ae7a0>
tarinfo:  <TarInfo 'kerasmodel.py' at 0x10c5aebb0>
tarinfo:  <TarInfo 'client1_new.py' at 0x10c5aec80>
tarinfo:  <TarInfo 'yolo3' at 0x10c5aeae0>
tarinfo:  <TarInfo 'yolo3/__init__.py' at 0x10c5aee20>
tarinfo:  <TarInfo 'yolo3/__pycache__' at 0x10c5aed50>
tarinfo:  <TarInfo 'yolo3/__pycache__/__init__.cpython-37.pyc' at 0x10d687050>
tarinfo:  <TarInfo 'yolo3/__pycache__/model.cpython-37.pyc' at 0x10d687120>
tarinfo:  <TarInfo 'yolo3/__pycache__/utils.cpython-37.pyc' at 0x10c5aeef0>
tarinfo:  <TarInfo 'yolo3/model.py' at 0x10d6871f0>
tarinfo:  <TarInfo 'yolo3/utils.py' at 0x10d687390>


SyntaxError: unexpected EOF while parsing (kerasmodel.py, line 3)

In [ ]:
model = create_seed_model()

In [ ]:
import crazy_test
sys.path.append('/Users/mattiasakesson/Documents/projects/fedn/test/mnist/seed/gamsel')
crazy_test.jumpboy()

In [ ]:
import sys
sys.path.append('/Users/mattiasakesson/Documents/projects/fedn/test/mnist/seed/gamsel')
import gamsel.life as life

In [1]:
from kerasmodel import create_seed_model

ModuleNotFoundError: No module named 'kerasmodel'

In [23]:
import tarfile
import sys
import os
path = '/Users/mattiasakesson/Documents/projects/fedn/test/fedbird/package/client'
_, tempf = tempfile.mkstemp()
tar = tarfile.open(tempf, "w:gz")
print("folder: ", path)
for filename in os.listdir(path):
    #if filename.endswith(".py"):
    print("   filename: ", filename)
    tar.add(os.path.join(path,filename), filename)
    if os.path.isdir(os.path.join(path,filename)):
        #tar.add(os.path.join(path,filename), filename)
        print("   folder: ", filename)

        for innerfilename in os.listdir(os.path.join(path,filename)):
            #if filename.endswith(".py"):
            print("      filename: ", innerfilename)
            tar.add(os.path.join(path,filename,innerfilename), os.path.join(filename,innerfilename))
            

        

tar.close()

folder:  /Users/mattiasakesson/Documents/projects/fedn/test/fedbird/package/client
   filename:  misc.py
   filename:  client1.py
   filename:  server.py
   filename:  yolo3
   folder:  yolo3
      filename:  .DS_Store
      filename:  model.py
      filename:  utils.py
   filename:  .DS_Store
   filename:  requirements.txt
   filename:  model_data
   folder:  model_data
      filename:  yolov3-tiny.h5
      filename:  yolov3-tiny.cfg
      filename:  seabird_classes.txt
      filename:  yolo_anchors.txt
      filename:  tiny_yolo_anchors.txt
   filename:  convert.py
   filename:  kerasmodel.py
   filename:  train.py
   filename:  inference.py
   filename:  yolo.py
   filename:  client1_new.py
   filename:  client2.py


In [24]:
from io import BytesIO
a = BytesIO()
a.seek(0, 0)
with open(tempf, 'rb') as f:
    a.write(f.read())
os.unlink(tempf)
package = a

In [26]:
_, path = tempfile.mkstemp()
with open(path, 'wb') as fh:
    fh.write(package.getbuffer())
    fh.flush()
# unzip
print("unzip")
tempf = tempfile.TemporaryDirectory()
tar = tarfile.open(path, "r:gz")
for tarinfo in tar:
    print("tarinfo: ", tarinfo)
    tar.extract(tarinfo, tempf.name)
tar.close()
print("list unziped files")
print("folder: ", tempf.name)
for filename in os.listdir(tempf.name):
    print("    filename: ", filename)
    if os.path.isdir(os.path.join(tempf.name,filename)):
        for innerfilename in os.listdir(os.path.join(tempf.name,filename)):
            print("      filename: ", innerfilename)
    

unzip
tarinfo:  <TarInfo 'misc.py' at 0x10e354d50>
tarinfo:  <TarInfo 'client1.py' at 0x10e175e20>
tarinfo:  <TarInfo 'server.py' at 0x10e3df1f0>
tarinfo:  <TarInfo 'yolo3' at 0x10e36c600>
tarinfo:  <TarInfo 'yolo3/.DS_Store' at 0x10e36c120>
tarinfo:  <TarInfo 'yolo3/model.py' at 0x10e36c530>
tarinfo:  <TarInfo 'yolo3/utils.py' at 0x10e36c6d0>
tarinfo:  <TarInfo 'yolo3/.DS_Store' at 0x10e3f8120>
tarinfo:  <TarInfo 'yolo3/model.py' at 0x10e3f82c0>
tarinfo:  <TarInfo 'yolo3/utils.py' at 0x10e3f8390>
tarinfo:  <TarInfo '.DS_Store' at 0x10e3f8ae0>
tarinfo:  <TarInfo 'requirements.txt' at 0x10e3f8530>
tarinfo:  <TarInfo 'model_data' at 0x10e3f8050>
tarinfo:  <TarInfo 'model_data/seabird_classes.txt' at 0x10e3f81f0>
tarinfo:  <TarInfo 'model_data/tiny_yolo_anchors.txt' at 0x10e3f8460>
tarinfo:  <TarInfo 'model_data/yolo_anchors.txt' at 0x10e3f8600>
tarinfo:  <TarInfo 'model_data/yolov3-tiny.cfg' at 0x10e3f86d0>
tarinfo:  <TarInfo 'model_data/yolov3-tiny.h5' at 0x10e3f87a0>
tarinfo:  <TarInfo

In [27]:
path = '/Users/mattiasakesson/Documents/projects/fedn/test/fedbird/package/client'
for root, dirs, files in os.walk(path):
    print("root: ", root, ", dirs: ", dirs, ", files: ", files)

root:  /Users/mattiasakesson/Documents/projects/fedn/test/fedbird/package/client , dirs:  ['yolo3', 'model_data'] , files:  ['misc.py', 'client1.py', 'server.py', '.DS_Store', 'requirements.txt', 'convert.py', 'kerasmodel.py', 'train.py', 'inference.py', 'yolo.py', 'client1_new.py', 'client2.py']
root:  /Users/mattiasakesson/Documents/projects/fedn/test/fedbird/package/client/yolo3 , dirs:  [] , files:  ['.DS_Store', 'model.py', 'utils.py']
root:  /Users/mattiasakesson/Documents/projects/fedn/test/fedbird/package/client/model_data , dirs:  [] , files:  ['yolov3-tiny.h5', 'yolov3-tiny.cfg', 'seabird_classes.txt', 'yolo_anchors.txt', 'tiny_yolo_anchors.txt']


In [30]:
tar = tarfile.open('newtar.tar.gz', "w:gz")
        
tar.add(path,"newtar")
tar.close()

In [34]:
tar = tarfile.open('newtar.tar.gz', "r:gz")
tar.extractall()